In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from tqdm import tqdm
import os
import time

In [1]:
# Define EEG processing parameters
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] # 14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] # 5 bands
window_size = 256 # Averaging band power of 2 sec
step_size = 16 # Each 0.125 sec update once
sample_rate = 128 # Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']#,'21','22','23','24','25','26','27','28','29','30','31','32'] # List of subjects


In [4]:
# Load and process data
data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

for subjects in subjectList:
    file_path = os.path.join('out', f's{subjects}.npy')
    
    with open(file_path, 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        for i in range(sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            elif i % 8 == 1:
                data_validation.append(sub[i][0])
                label_validation.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])

In [5]:
np.save(os.path.join('out', 'data_training'), np.array(data_training), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_training'), np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save(os.path.join('out', 'data_testing'), np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_testing'), np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

np.save(os.path.join('out', 'data_validation'), np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_validation'), np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)



training dataset: (292800, 70) (292800, 4)
testing dataset: (48800, 70) (48800, 4)
validation dataset: (48800, 70) (48800, 4)


In [6]:
# Loading training data
with open('out/data_training.npy', 'rb') as fileTrain:
    X_train = np.load(fileTrain)

with open('out/label_training.npy', 'rb') as fileTrainL:
    Y_train = np.load(fileTrainL)

# Loading validation data
with open('out/data_validation.npy', 'rb') as fileValid:
    X_val = np.load(fileValid)

with open('out/label_validation.npy', 'rb') as fileValidL:
    Y_val = np.load(fileValidL)


In [7]:
print("Normalizing data...")
X_train_normalized = normalize(X_train)
X_val_normalized = normalize(X_val)

# Extracting individual labels
Valence_train = np.ravel(Y_train[:, [1]])
Arousal_train = np.ravel(Y_train[:, [0]])
Dominance_train = np.ravel(Y_train[:, [2]])
Liking_train = np.ravel(Y_train[:, [3]])

Valence_val = np.ravel(Y_val[:, [1]])
Arousal_val = np.ravel(Y_val[:, [0]])
Dominance_val = np.ravel(Y_val[:, [2]])
Liking_val = np.ravel(Y_val[:, [3]])





Normalizing data...


Scaling 

In [8]:
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)

Valence

In [10]:

knn_valence = KNeighborsRegressor(n_neighbors=7, weights="uniform", p=1, algorithm='brute')
knn_valence.fit(X_train_normalized, Valence_train)
valence_predictions = knn_valence.predict(X_val_normalized)
valence_mse = mean_squared_error(Valence_val, valence_predictions)
print("Valence - Validation Mean Squared Error (MSE):", valence_mse)


Valence - Validation Mean Squared Error (MSE): 1.3691196316075611


Arousal

In [11]:

# KNN Model for Arousal
knn_arousal = KNeighborsRegressor(n_neighbors=7, weights='uniform',p=1,algorithm='brute')
knn_arousal.fit(X_train_normalized, Arousal_train)
arousal_predictions = knn_arousal.predict(X_val_normalized)
arousal_mse = mean_squared_error(Arousal_val, arousal_predictions)
print("Arousal - Validation Mean Squared Error (MSE):", arousal_mse)


Arousal - Validation Mean Squared Error (MSE): 1.6473423354382737


Dominance

In [12]:

# KNN Model for Dominance
knn_dominance = KNeighborsRegressor(n_neighbors=7, weights='uniform',p=1, algorithm='brute')
knn_dominance.fit(X_train_normalized, Dominance_train)
dominance_predictions = knn_dominance.predict(X_val_normalized)
dominance_mse = mean_squared_error(Dominance_val, dominance_predictions)
print("Dominance - Validation Mean Squared Error (MSE):", dominance_mse)


Dominance - Validation Mean Squared Error (MSE): 1.457959044998327


Liking

In [17]:

# KNN Model for Liking
knn_liking = KNeighborsRegressor(n_neighbors=7, weights='distance',p=1, algorithm='brute')
knn_liking.fit(X_train_normalized, Liking_train)
liking_predictions = knn_liking.predict(X_val_normalized)
liking_mse = mean_squared_error(Liking_val, liking_predictions)
print("Liking - Validation Mean Squared Error (MSE):", liking_mse)


Liking - Validation Mean Squared Error (MSE): 1.4514042467385733


In [18]:
from sklearn.metrics import accuracy_score, mean_squared_error, f1_score, mean_absolute_error
import numpy as np

# Function to binarize labels based on the median value as a threshold
def binarize_labels(labels, threshold=None):
    if threshold is None:
        threshold = np.median(labels)
    return np.where(labels >= threshold, 1, 0)

# Function to calculate and print metrics
def calculate_and_print_metrics(true_labels, predictions, feature_name):
    mse = mean_squared_error(true_labels, predictions)
    mae = mean_absolute_error(true_labels, predictions)
    threshold = np.median(true_labels)  # Using median of true labels as threshold
    true_classes = binarize_labels(true_labels, threshold)
    predicted_classes = binarize_labels(predictions, threshold)
    
    accuracy = accuracy_score(true_classes, predicted_classes)
    f1score = f1_score(true_classes, predicted_classes, average='weighted')
    std_dev = np.std(np.abs(true_labels - predictions))
    
    print(f"{feature_name} - Validation Mean Squared Error (MSE): {mse}")
    print(f"{feature_name} - Validation Mean Absolute Error (MAE): {mae}")
    print(f"{feature_name} - Validation Accuracy: {accuracy}")
    print(f"{feature_name} - Validation F1-score: {f1score}")
    print(f"{feature_name} - Validation Std_dev: {std_dev}")
    print()

# Calculate and print metrics for all features
print("Calculating metrics for Liking...")
calculate_and_print_metrics(Liking_val, liking_predictions, "Liking")

print("Calculating metrics for Dominance...")
calculate_and_print_metrics(Dominance_val, dominance_predictions, "Dominance")

print("Calculating metrics for Arousal...")
calculate_and_print_metrics(Arousal_val, arousal_predictions, "Arousal")

print("Calculating metrics for Valence...")
calculate_and_print_metrics(Valence_val, valence_predictions, "Valence")


Calculating metrics for Liking...
Liking - Validation Mean Squared Error (MSE): 1.4514042467385733
Liking - Validation Mean Absolute Error (MAE): 0.7670405995878238
Liking - Validation Accuracy: 0.8337909836065573
Liking - Validation F1-score: 0.832933434012569
Liking - Validation Std_dev: 0.9290064398713956

Calculating metrics for Dominance...
Dominance - Validation Mean Squared Error (MSE): 1.457959044998327
Dominance - Validation Mean Absolute Error (MAE): 0.8090478337236535
Dominance - Validation Accuracy: 0.825266393442623
Dominance - Validation F1-score: 0.8254074903906702
Dominance - Validation Std_dev: 0.8963261949454512

Calculating metrics for Arousal...
Arousal - Validation Mean Squared Error (MSE): 1.6473423354382737
Arousal - Validation Mean Absolute Error (MAE): 0.8801151932084309
Arousal - Validation Accuracy: 0.8212090163934426
Arousal - Validation F1-score: 0.8209727754751492
Arousal - Validation Std_dev: 0.9342053211804994

Calculating metrics for Valence...
Valence 